In [ ]:
# import packages
using DigitalMusicology
using DataFrames
using Plots

# 1) Convert Frequencies to Pitches and vice verca

In [ ]:
# A4 is set to 440Hz
frequency_to_pitch(f) = 69 + 12 * log2(f / 440)

pitch_to_frequency(p) = 2^((p - 69) / 12) * 440

In [ ]:
# the frequency of C4
pitch_to_frequency(60)

In [ ]:
# the lowest midi note has a frequency of about 8Hz,
# the highest about 13kHz
# humans typically hear between 20Hz and 20kHz
pitch_to_frequency(0), pitch_to_frequency(127)

# 2) Example of this Weeks Tutorial: Iron Maiden *Run to the hills*
[song on youtube](https://www.youtube.com/watch?v=3ZlDZPYzfm4)

[find midi file here](http://www.maidenmidi.com/number.html)

The midi file can also be read by Garageband and MuseScore!

# 3) Read MIDI File as a DataFrame
- A MIDI file is essentially a list of **MIDI events / MIDI commands**
- There are of are **note-on**, **note-off**, and **meta events** grouped by **channels** (repesenting voices) that are grouped by **tracks** (representing instruments)
- For note events, **pitch** and **velocity** are specified
- Meta events e.g. specify the key of a piece or indicate tempo changes
- To work with a MIDI file, we convert it into a list of notes

In [ ]:
notes = midifilenotes("Run_To_The_Hills.mid")
head(notes)

- `key_sharps` and `key_major` have default values
- Be careful if you see the default setting `(0,true)` like here!

In [ ]:
notes[:key_sharps] |> unique

In [ ]:
?midifilenotes

In [ ]:
# this is a small pipeline
notes[:track] |> unique |> sort

In [ ]:
# it is equivalent to
sort(unique(notes[:track]))

In [ ]:
track_names = ["meta", "vocals", "vocal harmony", "guitar 1", "guitar 2", "bass", "drums"]

- Track 1 is reserved for meta events!
- The other tracks are for vocals, vocal harmony, guitar 1, guitar 2, bass, drums
- one row in the data frame represents one note
- the columns are the features that we know about the notes

# 4) A look at the first bar

In [ ]:
# drums in first bar
# there are 192 ticks per quarter note
notes[1:20, [:onset_wholes, :onset_ticks]]

# Exploratory Analysis
- Pitch histogram
- Pitch class histogram
- Pitch class histogram per instrument
- Note duration histogram per instrument
- Beat histogram (onsets per beat) per instrument

# 5) Plot a pitch histogram

In [ ]:
# select the the instruments other than the drums
not_percussive_notes = notes[notes[:track] .!= 7, :]
head(not_percussive_notes)

In [ ]:
# convert midi notes to pitch numbers
pitches = [note.pitch for note in not_percussive_notes[:pitch]]
pitches[1:10]

In [ ]:
# plot pitch histogram
histogram(pitches, bins=collect(0:127))

# 6) Plot a pitch class histogram

In [ ]:
# plot pitch class histogram
pitch_classes = [mod(p, 12) for p in pitches]
xticks = (collect(0:11) .+ 0.5, collect(0:11))
histogram(pitch_classes, bins=12, xticks=xticks)

Looks like a C major scale!

In [ ]:
# shift the histogram so that the most prominent note is in front
shifted_xticks = (collect(0:11) .+ 0.5, [mod(pc+7, 12) for pc in 0:11])
histogram(
    [mod(pc-7, 12) for pc in pitch_classes], 
    bins=12, 
    xticks=shifted_xticks
)

In [ ]:
transform(p) = mod((p+4)*7, 12)
histogram(
    transform.(pitch_classes), 
    bins   = 12, 
    xticks = (transform.(collect(0:11)) .+ 0.5, (collect(0:11))), 
    xlim   = (0,12)
)

# 7) Plot a Pitch Class Histogram per Instrument

In [ ]:
for track in 2:6
    display(
        histogram(
            [mod(note.pitch, 12) for note in notes[notes[:track] .== track, :pitch]], 
            bins   = collect(0:12),
            xticks = xticks,
            title  = track_names[track],
            xlim = (0,12)
        )
    )
end

# 8) Plot a Note Duration Histogram per Instrument

In [ ]:
onsets  = notes[notes[:track] .== 4, :onset_wholes]
offsets = notes[notes[:track] .== 4, :offset_wholes]
durations = offsets - onsets

In [ ]:
[(d.num + 1) // d.den for d in durations]

In [ ]:
for track in 2:6
    onsets  = notes[notes[:track] .== track, :onset_wholes]
    offsets = notes[notes[:track] .== track, :offset_wholes]
    durations = offsets - onsets
    display(histogram([(d.num + 1) // d.den for d in durations], title=track_names[track]))
end

# 9) Plot beat histogram (onsets per beat) per instrument

In [ ]:
for track in 2:6
    onsets  = notes[notes[:track] .== track, :onset_wholes]
    display(
        histogram(
            [o - floor(o) for o in onsets], 
            bins  = [x*1/16 for x in 0:16], 
            title = track_names[track]
        )
    )
end